# HW3 Q4 [10 pts]



## Important Notices

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> add any cells to this Jupyter Notebook, because that will crash the autograder.
</div>


All instructions, code comments, etc. in this notebook **are part of the assignment instructions**. That is, if there is instructions about completing a task in this notebook, that task is not optional.  



<div class="alert alert-block alert-info">
    You <strong>must</strong> implement the following functions in this notebook to receive credit.
</div>

`user()`

`load_data()`

`exclude_no_pickuplocations()`

`exclude_no_tripdistance()`

`include_fare_range()`

`get_highest_tip()`

`get_total_toll()`

Each method will be auto-graded using different sets of parameters or data, to ensure that values are not hard-coded.  You may assume we will only use your code to work with data from NYC Taxi Trips during auto-grading. You do not need to write code for unreasonable scenarios.  

Since the overall correctness of your code will require multiple function to work together correctly (i.e., all methods are interdepedent), implementing only a subset of the functions likely will lead to a low score.

### Helper functions

You are permitted to write additional helper functions, or use additional instance variables so long as the previously described functions work as required.  

#### Pyspark Imports
<span style="color:red">*Please don't modify the below cell*</span>

In [1]:
import pyspark
from pyspark.sql import SQLContext
from pyspark.sql import *
import pyspark.sql.functions as func

#### Define Spark Context
<span style="color:red">*Please don't modify the below cell*</span>

In [2]:
sc
sqlContext = SQLContext(sc)

### Student Section - Please compete all the functions below

#### Function to return GT Username

In [3]:
def user():
        """
        :return: string
        your GTUsername, NOT your 9-Digit GTId  
        """         
        return 'zpatwary3'

#### Function to load data

In [4]:
def load_data(gcp_storage_path):
    """
        :param gcp_storage_path: string (full gs path including file name e.g gs://bucket_name/data.csv) 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to load yellow_tripdata_2019-01.csv data from your GCP storage bucket#
    #                                                              #        
    ################################################################
    # df = spark.read.csv("gs://zpatwary3/yellow_tripdata_2019-01.csv")
    df = sqlContext.read.option("header",True).csv("gs://zpatwary3/yellow_tripdata_2019-01.csv")
    
    df = df.withColumn('trip_distance',df['trip_distance'].cast('Decimal(38,2)'))
    df = df.withColumn('fare_amount',df['fare_amount'].cast('Decimal(38,2)'))
    df = df.withColumn('tip_amount',df['tip_amount'].cast('Decimal(38,2)'))
    df = df.withColumn('tolls_amount',df['tolls_amount'].cast('Decimal(38,2)'))
    df.show(5)
    return df

#### Function to exclude trips that don't have a pickup location

In [36]:
def exclude_no_pickuplocations(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no pickup locations               #
    # Note: Exclude  nulls, blanks and zeros                       #        
    ################################################################
    df = df.filter((df.PULocationID != None) | (df.PULocationID != ''))
     
    return df

#### Function to exclude trips with no distance

In [37]:
def exclude_no_tripdistance(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to exclude trips with no trip distances                 #
    # Note: Exclude  nulls, blanks and zero                        #        
    ################################################################
    df = df.filter((df.trip_distance != None) | (df.trip_distance != 0.00) | (df.trip_distance != ''))
    #df.show(20)
    return df

#### Function to include fare amount between the range of 20 to 60 Dollars

In [17]:
def include_fare_range(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: spark dataframe  
    """
    
    ################################################################
    # code to include trips with only within the fare range of     #
    # 20 to 60 dollars (including 20 and 60 dollars)               #        
    ################################################################
    df = df.filter((df.fare_amount >= 20) & (df.fare_amount <= 60))
    #df.show(20)
    return df

#### Function to get the highest tip amount

In [48]:
def get_highest_tip(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get the highest tip                                  #
    #                                                              #        
    ################################################################

    max_tip = df.agg({"tip_amount": "max"}).collect()[0][0]
    #max_tip = spark.sql("SELECT MAX(tip_amount) as max_tip FROM df").first().asDict()['max_tip']
    return max_tip

#### Function to get total toll amount

In [9]:
def get_total_toll(df):
    """
        :param nyc tax trips dataframe: spark dataframe 
        :return: decimal (rounded to 2 digits)  (NOTE: DON'T USE FLOAT)
    """
    
    ################################################################
    # code to get total toll amount                                #
    #                                                              #        
    ################################################################
    df = df.withColumn("tolls_amount", df["tolls_amount"].cast("decimal(38,2)"))
    total_toll = df.agg({"tolls_amount": "sum"}).collect()[0][0]
    return total_toll

### Run above functions and print

#### Uncomment the cells below and test your implemented functions

#### Load data from yellow_tripdata_2019-01.csv 

In [10]:
gcp_storage_path = "gs://zpatwary3/yellow_tripdata_2019-01.csv"
df = load_data(gcp_storage_path)
df.printSchema()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|         1.50|         1|                 N|         151|         239|           1|       7.00|  0.5|    0.5|      1.65|        0.00|                  0.3

#### Print total numbers of rows in the dataframe

In [11]:
df.count()

7667792

#### Print total number of rows in the dataframe after excluding trips with no pickup location

In [38]:
df_no_pickup_locations = exclude_no_pickuplocations(df)
df_no_pickup_locations.count()

7667792

#### Print total number of rows in the dataframe after exclude trips with no distance

In [39]:
df_no_trip_distance = exclude_no_tripdistance(df)
df_no_trip_distance.count()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       1| 2019-01-01 00:46:40|  2019-01-01 00:53:20|              1|         1.50|         1|                 N|         151|         239|           1|       7.00|  0.5|    0.5|      1.65|        0.00|                  0.3

7613022

#### Print total number of rows in the dataframe after including trips with fair amount between the range of 20 to 60 Dollars

In [18]:
df_include_fare_range = include_fare_range(df)
df_include_fare_range.count()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+
|       2| 2018-11-28 15:56:57|  2018-11-28 15:58:33|              5|         0.00|         2|                 N|         193|         193|           2|      52.00|    0|    0.5|      0.00|        0.00|                  0.3

969417

#### Print the highest tip amount

In [49]:
max_tip = get_highest_tip(df)
print(max_tip)

787.25


#### Print the total toll amount

In [16]:
total_toll = get_total_toll(df)
print(total_toll)

2430066.70
